In [13]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.metrics import accuracy_score, mean_absolute_error, r2_score, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.model_selection import TimeSeriesSplit
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import ParameterGrid
import joblib

First, we’ll identify a target that we’re trying to predict. Our target will be if the next close price will go up or down tomorrow. If the price went up, the target will be 1.0, and if it went down, the target will be 0.0.

EMA explanation

In [166]:
import yfinance as yf
def prepare_stock_data(stock, start_date='2018-01-01', end_date='2021-01-18'):
    #df = pd.read_json('../data/historical_data/' + stock + '.json', lines=True)
    #df1 = pd.read_json('../data/historical_data/S&P500.json', lines=True)

    
    df = yf.download(stock, start=start_date, end=end_date)
    df1 = yf.download('SPY', start=start_date, end=end_date)
    
    df.reset_index(inplace=True)
    df1.reset_index(inplace=True)
    
    df1 = df1.rename(columns={'Open': 'S&P 500 Open', 'Close': 'S&P 500 Close'})

    # Join dataframes
    df = pd.merge(df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']],
                  df1[['Date', 'S&P 500 Open', 'S&P 500 Close']],
                  on='Date', how='outer')
    
    # Set date as index
    df['Date'] = df['Date'].astype(str).str.split(' ').str[0]
    df = df.set_index('Date')
    
    # Add label
    df['Label'] = df.rolling(2).apply(lambda x: x.iloc[1] > x.iloc[0])['Close']
    
    # Shift one day, we can not use the future to predict the past
    df[['Open', 'High', 'Low', 'Close', 'Volume', 'S&P 500 Open', 'S&P 500 Close']] = df[['Open', 'High', 'Low', 'Close', 'Volume', 'S&P 500 Open', 'S&P 500 Close']].shift(1)
    
    df = df.rename(columns={'Open': 'Prev Open', 'High': 'Prev High', 'Low': 'Prev Low', 
                            'Close': 'Prev Close', 'Volume': 'Prev Volume', 'S&P 500 Open': 'Prev S&P 500 Open', 
                            'S&P 500 Close': 'Prev S&P 500 Close'})
    
    
    # Add sentiment analysis
    
    # Compute Exponential Mobile Average (EMA) for stock close prices
    delta = df['Prev Close'] - df['Prev Open']
    df['10 Days EMA'] = np.round(delta.copy().ewm(span=10, adjust=False).mean(), decimals=3)
    df['5 Days EMA'] = np.round(delta.copy().ewm(span=5, adjust=False).mean(), decimals=3)
    df['3 Days EMA'] = np.round(delta.copy().ewm(span=3, adjust=False).mean(), decimals=3)

    delta = df['Prev S&P 500 Close'] - df['Prev S&P 500 Open']
    df['S&P 500 10 Days EMA'] = np.round(delta.copy().ewm(span=10, adjust=False).mean(), decimals=3)
    df['S&P 500 5 Days EMA'] = np.round(delta.copy().ewm(span=5, adjust=False).mean(), decimals=3)
    df['S&P 500 3 Days EMA'] = np.round(delta.copy().ewm(span=3, adjust=False).mean(), decimals=3)

    # Drop rows with NaN values
    df.dropna(inplace=True)

    # Re order columns
    #df = df[['Date', 'Stock Trend EMA', 'S&P 500 Trend EMA', 'Label']]

    return df

df = prepare_stock_data('SPY')
#df.append(prepare_stock_data('AMZN'), ignore_index=True)
#df.append(prepare_stock_data('GOOGL'), ignore_index=True)
#df.append(prepare_stock_data('TSLA'), ignore_index=True)
#df.append(prepare_stock_data('MSFT'), ignore_index=True)
#df.append(prepare_stock_data('AAPL'), ignore_index=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Prev Open,Prev High,Prev Low,Prev Close,Prev Volume,Prev S&P 500 Open,Prev S&P 500 Close,Label,10 Days EMA,5 Days EMA,3 Days EMA,S&P 500 10 Days EMA,S&P 500 5 Days EMA,S&P 500 3 Days EMA
0,267.839996,268.809998,267.399994,268.769989,86655700.0,267.839996,268.769989,1.0,0.930,0.930,0.930,0.930,0.930,0.930
1,268.959991,270.640015,268.959991,270.470001,90070400.0,268.959991,270.470001,1.0,1.035,1.123,1.220,1.035,1.123,1.220
2,271.200012,272.160004,270.540009,271.609985,80636400.0,271.200012,271.609985,1.0,0.922,0.886,0.815,0.922,0.886,0.815
3,272.510010,273.559998,271.950012,273.420013,83524000.0,272.510010,273.420013,1.0,0.920,0.894,0.862,0.920,0.894,0.862
4,273.309998,274.100006,272.980011,273.920013,57319200.0,273.309998,273.920013,1.0,0.863,0.799,0.736,0.863,0.799,0.736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1525,132.429993,132.630005,130.229996,132.050003,105158200.0,380.589996,381.260010,0.0,-0.138,0.079,0.271,0.883,1.396,1.585
1526,129.190002,130.169998,128.500000,128.979996,100384500.0,377.850006,378.690002,0.0,-0.151,-0.018,0.031,0.875,1.210,1.213
1527,128.500000,129.690002,126.860001,128.800003,91951100.0,378.890015,378.769989,1.0,-0.069,0.088,0.165,0.694,0.767,0.546
1528,128.759995,131.449997,128.490005,130.889999,88636800.0,378.690002,379.790009,0.0,0.331,0.769,1.148,0.768,0.878,0.823


the training and test set have to follow chronological order.

In [167]:
predictors = [#'Prev Open', NO
              #'Prev High', NO
              #'Prev Low', NO
              'Prev Close', # SI
              'Prev Volume', # SI
              '10 Days EMA', # SI
              '5 Days EMA', # SI
              '3 Days EMA', # SI
              #'S&P 500 10 Days EMA', 
              #'S&P 500 5 Days EMA', 
              #'S&P 500 3 Days EMA',
              #'Prev S&P 500 Open',
              #'Prev S&P 500 Close'
            ]
x_train, x_test, y_train, y_test = train_test_split(df[predictors],
                                                    df[['Label']], test_size=.2,
                                                    shuffle=True, random_state=0)

print('Size of train set: ', x_train.shape)
print('Size of test set: ', x_test.shape)
print('Size of train set: ', y_train.shape)
print('Size of test set: ', y_test.shape)

Size of train set:  (612, 5)
Size of test set:  (153, 5)
Size of train set:  (612, 1)
Size of test set:  (153, 1)


In [168]:
grid = {'n_estimators': [200], 'max_depth': [3], 'max_features': [1, 8], 'random_state': [42]}
test_scores = []

rf_model = RandomForestRegressor()

for g in ParameterGrid(grid):
    rf_model.set_params(**g) 
    rf_model.fit(x_train, y_train.values.ravel())
    test_scores.append(rf_model.score(x_test, y_test))

best_index = np.argmax(test_scores)
print(test_scores[best_index], ParameterGrid(grid)[best_index])

ValueError: max_features must be in (0, n_features]

In [169]:
# Spot-Check Algorithms
classifiers = [
    RandomForestClassifier(n_estimators=200, min_samples_split=200, max_features = 3, random_state=1, max_depth=3),
    XGBClassifier(),
    AdaBoostClassifier(),
    KNeighborsClassifier(3),
   # SVC(kernel="linear", C=0.025),
   # SVC(gamma=2, C=1),
    #GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5)
]

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ("pca", PCA())
])

numeric_features = predictors

preprocessor = ColumnTransformer(
   transformers=[
    ('numeric', numeric_transformer, numeric_features)
]) 


In [170]:
for classifier in classifiers:  
    
    pipe = Pipeline(steps = [
               ('preprocessor', preprocessor)
              ,('classifier', classifier)
           ])
    
    # Train the model
    pipe.fit(x_train, y_train.values.ravel())
    
    # Use model to make predictions
    y_pred = pipe.predict(x_test)
    
    # Evaluate the performance
    print("\nTraining ", classifier)
    accuracy = accuracy_score(y_pred, y_test)
    print("Accuracy on test set: ", accuracy)
    print("Metrics per class on test set:")

    print("Confusion matrix:")
    metrics.confusion_matrix(y_test, y_pred)

    print(metrics.classification_report(y_test, y_pred))


Training  RandomForestClassifier(max_depth=3, max_features=3, min_samples_split=200,
                       n_estimators=200, random_state=1)
Accuracy on test set:  0.5163398692810458
Metrics per class on test set:
Confusion matrix:
              precision    recall  f1-score   support

         0.0       0.31      0.06      0.10        69
         1.0       0.54      0.89      0.67        84

    accuracy                           0.52       153
   macro avg       0.42      0.48      0.38       153
weighted avg       0.43      0.52      0.41       153

[21:46:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Edoardo\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



Training  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Accuracy on test set:  0.5032679738562091
Metrics per class on test set:
Confusion matrix:
              precision    recall  f1-score   support

         0.0       0.44      0.35      0.39        69
         1.0       0.54      0.63      0.58        84

    accuracy                           0.50       153
   macro avg       0.49      0.49      0.4

In [147]:
# Save the Model to disk
filename = '../model/classification_model.pkl'
final_pipe = Pipeline(steps = [
               ('preprocessor', preprocessor)
              ,('classifier', classifiers[1])
           ])
joblib.dump(final_pipe, filename)

['../model/classification_model.pkl']

Evaluate the model using other test set

In [148]:
def predict_stock_value(stock):
    df = prepare_stock_data(stock)
    y_test = df['Label']
    
    # Load the Regression Model
    pipe = joblib.load('../model/classification_model.pkl')
    
    # Use model to make predictions
    y_pred = pipe.predict(df[predictors])

    # Evaluate the performance
    print("\n Evaluating ", pipe['classifier'])
    accuracy = accuracy_score(y_pred, y_test)
    print("Accuracy on test set: ", accuracy)
    print("Metrics per class on test set:")

    print("Confusion matrix:")
    metrics.confusion_matrix(y_test, y_pred)

    print(metrics.classification_report(y_test, y_pred))

In [149]:
predict_stock_value('TSLA')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

 Evaluating  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Accuracy on test set:  0.5187007874015748
Metrics per class on test set:
Confusion matrix:
              precision    recall  f1-score   support

         0.0       0.48      0.20      0.28       481
         1

In [150]:
predict_stock_value('AAPL')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

 Evaluating  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Accuracy on test set:  0.5295275590551181
Metrics per class on test set:
Confusion matrix:
              precision    recall  f1-score   support

         0.0       0.47      0.27      0.34       463
         1

In [152]:
predict_stock_value('GOOGL')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

 Evaluating  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Accuracy on test set:  0.5295275590551181
Metrics per class on test set:
Confusion matrix:
              precision    recall  f1-score   support

         0.0       0.47      0.18      0.26       466
         1

In [136]:
predict_stock_value('MSFT')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

 Evaluating  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Accuracy on test set:  0.5334645669291339
Metrics per class on test set:
Confusion matrix:
              precision    recall  f1-score   support

         0.0       0.45      0.24      0.32       448
         1